In [1]:
from random import sample, randint, random
from math import e
from statistics import mean, pstdev

## 80 points

In [2]:
def read_input(URL):
    with open(URL) as file_in:
        ti,tf = file_in.readline().split(' '); ti,tf = float(ti),float(tf)
        n = int(file_in.readline())
        w_limit = float(file_in.readline())
        objects = []
        for i in range(n):
            obj_v,obj_w = file_in.readline().split(' '); obj_v,obj_w = float(obj_v),float(obj_w)
            objects.append([obj_v,obj_w])
        return (ti,tf),n,w_limit,objects
read_input("example_input.txt")

((1000.0, 0.1),
 5,
 15.0,
 [[5.0, 2.0], [14.0, 3.0], [7.0, 7.0], [2.0, 5.0], [23.0, 10.0]])

In [3]:
(ti,tf),n,w_limit,objects = read_input("example_input.txt")

In [4]:
def w_value(p_solution,objects):
    w = 0
    for i,ele in enumerate(p_solution):
        if ele == 1:
            w += objects[i][1]
    return w

In [5]:
def p_value(p_solution,objects):
    w = 0
    for i,ele in enumerate(p_solution):
        if ele == 1:
            w += objects[i][0]
    return w

In [6]:
w_value([1,1,1,1,1],objects)

27.0

In [7]:
p_value([1,1,1,1,1],objects)

51.0

In [8]:
def initial_solution(n,w_limit,objects):
    obj_indexes = [i for i in range(n)]
    ini_sol = [0]*n
    rand_obj = -1
    while w_value(ini_sol,objects) <= w_limit: # stop when the solution weighs more than the limit
        rand_obj = sample(obj_indexes,1)[0]
        ini_sol[rand_obj] = 1
        obj_indexes.remove(rand_obj) # delete index so I won't pick it again next time
    ini_sol[rand_obj] = 0 
    return ini_sol, p_value(ini_sol,objects)#w_value(ini_sol,objects)

In [9]:
initial_solution(*read_input("example_input.txt")[1:])

([1, 0, 1, 1, 0], 14.0)

In [10]:
def bitflip(solution,i):
    new_sol = solution
    if new_sol[i] == 0:
        new_sol[i] = 1
    else:
        new_sol[i] = 0
    return new_sol    

In [11]:
print(bitflip([1, 1, 0, 1, 0],1))

[1, 0, 0, 1, 0]


In [12]:
'''
def neighborhood(solution,n,w_limit,objects):
    neighs = []
    n_neighs = 0
    for i in range(n):
        p_sol = bitflip(solution.copy(),i)
        p_sol_value = w_value(p_sol,objects)
        #print(p_sol)
        if p_sol_value <= w_limit:
            neighs.append(p_sol)#([p_sol,p_sol_value])
            n_neighs += 1
    return neighs, n_neighs
'''

'\ndef neighborhood(solution,n,w_limit,objects):\n    neighs = []\n    n_neighs = 0\n    for i in range(n):\n        p_sol = bitflip(solution.copy(),i)\n        p_sol_value = w_value(p_sol,objects)\n        #print(p_sol)\n        if p_sol_value <= w_limit:\n            neighs.append(p_sol)#([p_sol,p_sol_value])\n            n_neighs += 1\n    return neighs, n_neighs\n'

In [13]:
#neighborhood([1, 1, 0, 0, 1],n,w_limit,objects)

In [14]:
#def random_neighbor(neighs, n_neighs):
#    return neighs[randint(0,n_neighs-1)]

In [15]:
#random_neighbor(*neighborhood([1, 1, 0, 0, 1],n,w_limit,objects))

In [16]:
def rand_sol_from_neighborhood(solution,n,w_limit,objects):
    neighs = []
    n_neighs = 0
    n_indexes = [i for i in range(n)]
    while n_indexes != []:
        rand_index = sample(n_indexes,1)[0]
        p_sol = bitflip(solution.copy(),rand_index)
        p_sol_value = w_value(p_sol,objects)
        if p_sol_value <= w_limit:
            return p_sol
        n_indexes.remove(rand_index) # delete unusable index so I won't pick it again next time
    return []

In [17]:
rand_sol_from_neighborhood([1, 1, 0, 0, 1],n,w_limit,objects)

[1, 0, 0, 0, 1]

In [18]:
def index_list(l,n): # returns a list with the picked objects
    ind = []
    for i in range(n):
        if l[i] == 1:
            ind.append(i)
    return ind

In [39]:
index_list([1, 0, 0, 0, 1],5)

[0, 4]

In [19]:
def T(t):
    return 0.99*t

In [20]:
def simulated_annealing(ti_tf,n,w_limit,objects):
    (ti,tf) = ti_tf
    best_solution, best_solution_p = initial_solution(n,w_limit,objects)
    #print(best_solution,best_solution_p)
    t = T(ti)
    while t >= tf:
        aspirant_sol = rand_sol_from_neighborhood(best_solution,n,w_limit,objects)
        aspirant_sol_p = p_value(aspirant_sol,objects)
        if aspirant_sol_p >= best_solution_p: # here I changed it (<= to >=) to the opposite because the best value is a big one, not a small one.
            best_solution, best_solution_p = aspirant_sol, aspirant_sol_p
        elif random() < e**((-(best_solution_p-aspirant_sol_p))/t): # I swapped the p_values (aspirant_sol_p-best_solution_p to best_solution_p-aspirant_sol_p) because of the same reason stated 2 lines above
            #print(best_solution_p,aspirant_sol_p)
            #print(e**((-(aspirant_sol_p-best_solution_p))/t))
            best_solution, best_solution_p = aspirant_sol, aspirant_sol_p
        t = T(t)
    return index_list(best_solution,n),best_solution, best_solution_p, w_value(best_solution,objects)

In [21]:
simulated_annealing(*read_input("example_input.txt"))

([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0)

In [22]:
bck,bina,p,w = simulated_annealing(*read_input("example_input.txt"))
bina_t = str(bina).replace(', ','').replace(']','').replace('[','')
print(bck,bina_t)
print(p)
print(w)

[0, 1, 4] 11001
42.0
15.0


In [23]:
e

2.718281828459045

In [24]:
e**((-(5-1))/2)

0.1353352832366127

## 20 points

In [25]:
def m_simulated_annealings(m,ti_tf,n,w_limit,objects):
    m_solutions_and_scores = []
    for i in range(m):
        solution_nmore = simulated_annealing(ti_tf,n,w_limit,objects)
        m_solutions_and_scores.append(solution_nmore)
    return m_solutions_and_scores

In [26]:
m = 15
m_simulated_annealings(m,*read_input("example_input.txt"))

[([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 2], [1, 1, 1, 0, 0], 26.0, 12.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0),
 ([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0)]

In [27]:
def best_m_solution(m,m_solutions):
    best_index = 0
    best_index_score = m_solutions[0][2]
    for i in range(1,m):
        new_index_score = m_solutions[i][2]
        if new_index_score > best_index_score:
            best_index = i
            best_index_score = new_index_score
    return m_solutions[best_index]

In [28]:
m = 15
m_solutions = m_simulated_annealings(m,*read_input("example_input.txt"))
best_m_solution(m,m_solutions)

([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0)

In [29]:
def worst_m_solution(m,m_solutions):
    best_index = 0
    best_index_score = m_solutions[0][2]
    for i in range(1,m):
        new_index_score = m_solutions[i][2]
        if new_index_score < best_index_score:
            best_index = i
            best_index_score = new_index_score
    return m_solutions[best_index]

In [30]:
m = 15
m_solutions = m_simulated_annealings(m,*read_input("example_input.txt"))
worst_m_solution(m,m_solutions)

([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0)

In [31]:
def m_median(m,m_solutions):
    m_solutions.sort(key=lambda x:x[2])
    if m%2 == 0:
        return m_solutions[(m//2)-1] + m_solutions[(m//2)]
    else:
        return m_solutions[m//2]

In [32]:
m_median(m,m_solutions)

([0, 1, 4], [1, 1, 0, 0, 1], 42.0, 15.0)

In [33]:
def mean_score(m_solutions):
    # given m solutions, returns the mean score
    return mean([m[2] for m in m_solutions])

In [34]:
mean_score(m_solutions)

42.0

In [35]:
def std_score(m_solutions):
    # given m solutions, returns the standard deviation population score
    return pstdev([m[2] for m in m_solutions])

In [36]:
std_score(m_solutions)

0.0

In [37]:
def m_solutions_analysis(m,m_solutions):
    print('solutions analiyis (m = {}):'.format(m))
    print('Best solution:',*best_m_solution(m,m_solutions))
    print('Worst solution:',*worst_m_solution(m,m_solutions))
    print('Median solution:',*m_median(m,m_solutions))
    print('Mean p score:',mean_score(m_solutions))
    print('Standard deviation p score:',std_score(m_solutions))

In [38]:
m = 15
m_solutions = m_simulated_annealings(m,*read_input("example_input.txt"))
m_solutions_analysis(m,m_solutions)

solutions analiyis (m = 15):
Best solution: [0, 1, 4] [1, 1, 0, 0, 1] 42.0 15.0
Worst solution: [0, 1, 2] [1, 1, 1, 0, 0] 26.0 12.0
Median solution: [0, 1, 4] [1, 1, 0, 0, 1] 42.0 15.0
Mean p score: 40.93333333333333
Standard deviation p score: 3.991101212558871
